In [1]:
# initialize project:
using Pkg; Pkg.activate("./MyProject");;
using YAML

# Find current path
CURRENT = pwd(); LIB_PATH = joinpath(CURRENT,"libs");

# include auxiliary Julia scripts:
for file in readdir(LIB_PATH)
    # Check if the file ends with ".jl" extension
    if endswith(file, ".jl")
        include(joinpath(LIB_PATH, file))
    end
end

  Activating project at `~/Documents/GitHub/StabTheory/MyProject`


In [3]:
data = YAML.load_file("cnc.yaml")

Dict{Any, Any} with 4 entries:
  5 => Dict{Any, Any}(0=>Dict{Any, Any}("full_set"=>[[0, 1, 1, 0, 1, 0, 0, 0, 0…
  4 => Dict{Any, Any}(0=>Dict{Any, Any}("full_set"=>[[0, 0, 1, 0, 0, 0, 0, 0], …
  2 => Dict{Any, Any}(0=>Dict{Any, Any}("full_set"=>[[1, 1, 0, 0], [0, 1, 0, 0]…
  3 => Dict{Any, Any}(0=>Dict{Any, Any}("full_set"=>[[1, 1, 1, 0, 0, 0], [0, 0,…

In [3]:
# Generate the orbits for each n and m
orbits = Dict()
for n in 2:3
    n_orbits = Dict()
    for m in 0:n
        cnc = data[n][m]["full_set"]
        orbit = symplectic_orbit(n, cnc)
        orbit = [map(Vector, o) for o in orbit]
        orbit = [[map(Int, v) for v in o] for o in orbit]
        println("n=$n, m=$m: orbit size is $(length(orbit))")
        n_orbits["m=$m"] = orbit
    end
    orbits["n=$n"] = n_orbits
end

n=2, m=0: orbit size is 15
n=2, m=1: orbit size is 15
n=2, m=2: orbit size is 6
n=3, m=0: orbit size is 135
n=3, m=1: orbit size is 315
n=3, m=2: orbit size is 378
n=3, m=3: orbit size is 288


In [4]:
# Write the orbits to a YAML file
open("cnc_orbits.yaml", "w") do file
    YAML.write(file, orbits)
end

In [2]:
orbits = YAML.load_file("cnc_orbits.yaml")

cncs = Dict()
for n in 2:2
    n_cncs = Dict()
    for m in 0:n
        nm_cncs = Vector{Dict}()
        orbit = orbits["n=$n"]["m=$m"]
        for cnc in orbit
            d = Dict()
            cnc_set::Set{Vector{Int}} = Set(cnc)
            cnc = CncSet(cnc_set)
            isotropic_gens_list = [map(Int, v) for v in cnc.isotropic_gens]
            anticomm_paulis_list = [map(Int, v) for v in cnc.anticommuting_paulis]
            d["isotropic_gens"] = isotropic_gens_list
            d["anticommuting_paulis"] = anticomm_paulis_list
            push!(nm_cncs, d)
        end
        n_cncs["m=$m"] = nm_cncs
    end
    cncs["n=$n"] = n_cncs
end

In [3]:
println(cncs)

Dict{Any, Any}("n=2" => Dict{Any, Any}("m=2" => Dict[Dict{Any, Any}("anticommuting_paulis" => [[1, 1, 1, 0], [0, 1, 0, 1], [0, 0, 0, 1], [0, 1, 1, 0], [1, 1, 0, 0]], "isotropic_gens" => [[0, 0, 0, 0]]), Dict{Any, Any}("anticommuting_paulis" => [[1, 0, 1, 0], [0, 1, 1, 1], [0, 0, 1, 1], [0, 1, 1, 0], [1, 0, 0, 0]], "isotropic_gens" => [[0, 0, 0, 0]]), Dict{Any, Any}("anticommuting_paulis" => [[1, 0, 1, 0], [1, 1, 0, 1], [1, 0, 0, 1], [1, 1, 0, 0], [0, 0, 1, 0]], "isotropic_gens" => [[0, 0, 0, 0]]), Dict{Any, Any}("anticommuting_paulis" => [[0, 0, 0, 1], [0, 1, 0, 0], [0, 1, 1, 1], [1, 1, 0, 1], [1, 1, 1, 1]], "isotropic_gens" => [[0, 0, 0, 0]]), Dict{Any, Any}("anticommuting_paulis" => [[1, 0, 1, 1], [0, 1, 0, 1], [0, 1, 0, 0], [0, 0, 1, 1], [1, 0, 0, 1]], "isotropic_gens" => [[0, 0, 0, 0]]), Dict{Any, Any}("anticommuting_paulis" => [[1, 0, 1, 1], [1, 1, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [1, 1, 1, 1]], "isotropic_gens" => [[0, 0, 0, 0]])], "m=0" => Dict[Dict{Any, Any}("anticommuting_pa

In [4]:
open("cnc_by_orbits.yaml", "w") do file
    YAML.write(file, cncs)
end